In [ ]:
import os
from pymongo import MongoClient

### Connection to MongoDB

In [ ]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to database collections

In [ ]:
audio_descriptions = database['modified_audio_descriptions_x'] # last updated audio descriptions collection
audio_clips = database['audio_clips'] # last updated audio clips collection

##### Updating video field in audio descriptions collection based on audio descriptions field of video collection

In [ ]:
ad_pipeline = [
    {
        '$match': {
            'audio_clips': {
                '$exists': True, 
                '$ne': []
            }
        }
    }, {
        '$unwind': {
            'path': '$audio_clips'
        }
    }, {
        '$project': {
            '_id': 0, 
            'audio_clips_id': '$audio_clips', 
            'audio_description_id': '$_id'
        }
    }, {
        '$out': 'AC_AD'   # new collection
    }
]

audio_descriptions.aggregate(ad_pipeline)

In [ ]:
ac_pipeline = [
    {
        '$lookup': {
            'from': 'AC_AD', 
            'localField': '_id', 
            'foreignField': 'audio_clips_id', 
            'as': 'AC_AD'
        }
    }, {
        '$unwind': {
            'path': '$AC_AD'
        }
    }, {
        '$addFields': {
            'audio_description': '$AC_AD.audio_description_id'
        }
    }, {
        '$project': {
            'AC_AD': 0
        }
    }, {
        '$out': 'modified_audio_clips'
    }
]

audio_clips.aggregate(ac_pipeline)